# Data Loading and Cleaning

Loads and examines the initial structure of the datasets required.

In [234]:
import os
import pandas as pd
import seaborn as sns

In [235]:
# Load Data
df_adjusted = pd.read_csv('../data/combined_data_season_adjusted.csv')
df_not_adjusted = pd.read_csv('../data/combined_data_season_not_adjusted.csv')
categories = pd.read_csv('../data/code_categories.csv')

- **`df_adjusted` (*combined_data_season_adjusted*)**: Contains the main dataset with seasonally adjusted sales data for retail and food services from 2017–2023.
- 
**`df_not_adjusted` (*combined_data_season_not_adjusted*)**: Contains the main dataset with raw sales data for retail and food services from 2017–2023.
  
- **`categories` (*code_categories*)**: Provides a mapping of category codes to descriptive names, enabling a better understanding of each retail and food service segment.


In [236]:
pd.options.display.float_format = '{:.0f}'.format

In [237]:
df_adjusted.head()

,cell_value,time_slot_id,error_data,category_code,seasonally_adj,data_type_code,time,us
0,688810,0,no,44X72,yes,SM,2023-06,1
1,703256,0,no,44X72,yes,SM,2023-12,1
2,11124,0,no,4521E,yes,SM,2023-04,1
3,10871,0,no,4521E,yes,SM,2023-10,1
4,415563,0,no,44000,yes,SM,2017-02,1


In [238]:
categories.head()

,category_code,category_name,high_level_category
0,44X72,Retail Trade and Food Services,Retail and Food Services
1,4521E,Department Stores,Retail Trade
2,44000,Retail Trade,Retail Trade
3,444,Building Material and Garden Equipment and Sup...,Retail Trade
4,44Y72,"Retail Trade and Food Services, ex Auto",Retail and Food Services


In [239]:
df_adjusted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   cell_value      3500 non-null   float64
 1   time_slot_id    3500 non-null   int64  
 2   error_data      3500 non-null   object 
 3   category_code   3500 non-null   object 
 4   seasonally_adj  3500 non-null   object 
 5   data_type_code  3500 non-null   object 
 6   time            3500 non-null   object 
 7   us              3500 non-null   int64  
dtypes: float64(1), int64(2), object(5)
memory usage: 218.9+ KB


As the dataset contains no `null` values, we proceed to examine the unique values in each column to understand the variability and structure of the data.

In [240]:
for column in df_adjusted.columns:
    print(f"Column: {column}")
    print(f"Unique values: {df_adjusted[column].unique()}")
    print(f"Number of unique values: {df_adjusted[column].nunique()}\n")

Column: cell_value
Unique values: [6.88810e+05 7.03256e+05 1.11240e+04 ... 7.60000e+00 4.80000e+00
 5.00000e+00]
Number of unique values: 1797

Column: time_slot_id
Unique values: [0]
Number of unique values: 1

Column: error_data
Unique values: ['no' 'yes']
Number of unique values: 2

Column: category_code
Unique values: ['44X72' '4521E' '44000' '444' '44Y72' '451' '453' '4451' '441' '454'
 '441X' '443' '447' '446' '452' '448' '445' '44W72' '442' '44Z72' '722']
Number of unique values: 21

Column: seasonally_adj
Unique values: ['yes']
Number of unique values: 1

Column: data_type_code
Unique values: ['SM' 'E_SM']
Number of unique values: 2

Column: time
Unique values: ['2023-06' '2023-12' '2023-04' '2023-10' '2017-02' '2017-08' '2017-10'
 '2017-01' '2017-07' '2018-09' '2019-05' '2019-11' '2020-03' '2020-09'
 '2021-02' '2021-08' '2019-02' '2019-08' '2020-06' '2020-12' '2017-06'
 '2017-12' '2017-05' '2018-01' '2018-07' '2020-11' '2020-01' '2020-07'
 '2021-03' '2021-09' '2021-01' '2021-0

Convert data types to appropriate types

 -`time` to `Datetime` format for easier analysis.

In [241]:
df_adjusted['time'] = pd.to_datetime(df_adjusted['time'], format='%Y-%m')
df_not_adjusted['time'] = pd.to_datetime(df_not_adjusted['time'], format='%Y-%m')

Rename column names for better interpretation.

In [242]:
df_adjusted = df_adjusted.rename(columns={'seasonally_adj': 'seasonally_adjusted'})
df_not_adjusted = df_not_adjusted.rename(columns={'seasonally_adj': 'seasonally_adjusted'})
df_adjusted['seasonally_adjusted'] = df_adjusted['seasonally_adjusted'].map({'yes': True, 'no': False})
df_not_adjusted['seasonally_adjusted'] = df_not_adjusted['seasonally_adjusted'].map({'yes': True, 'no': False})

Proceed to drop unnecessary columns.

In [243]:
df_adjusted = df_adjusted.drop(columns=['time_slot_id', 'us'])
df_not_adjusted = df_not_adjusted.drop(columns=['time_slot_id', 'us'])

Merge the main dataset with a mapping dataset (categories) using the `category_code` column. 

This step assigns descriptive names to each `category_code`, making the dataset more interpretable.

In [244]:
df_adjusted = df_adjusted.merge(categories, on='category_code', how='left')
df_not_adjusted = df_not_adjusted.merge(categories, on='category_code', how='left')

Inspect `error_data` and `cell_value` for the two unique `data_type_code`s.

In [245]:
# Group by 'data_type_code' and get unique values of 'error_data' and basic stats for 'cell_value'
result = data.groupby('data_type_code').agg({
    'error_data': 'unique',     
    'cell_value': ['unique', 'count', 'min', 'max'] 
})

result

error_data                                         cell_value  \
                   unique                                             unique   
data_type_code                                                                 
E_SM                [yes]  [0.9, 0.7, 0.0, 0.6, 1.9, 2.3, 0.5, 1.2, 2.5, ...   
SM                   [no]  [688810.0, 703256.0, 11124.0, 10871.0, 415563....   

                                  
               count  min    max  
data_type_code                    
E_SM            1736    0      8  
SM              1764 2885 703256

`SM (Sales Monthly)`: Represents raw sales figures.

`E_SM (Sales Monthly Percent Change)`: Reflects percentage changes in sales.

Due to their differing scales and contexts, separating these datasets ensures more accurate and focused analysis.

In [246]:
df_adjusted = df_adjusted.drop(columns=['error_data'])
df_not_adjusted = df_not_adjusted.drop(columns=['error_data'])

In [247]:
sm_df_adjusted = df_adjusted[df_adjusted['data_type_code'] == 'SM']
esm_df_adjusted = df_adjusted[df_adjusted['data_type_code'] == 'E_SM']
sm_df_not_adjusted = df_not_adjusted[df_not_adjusted['data_type_code'] == 'SM']
esm_df_not_adjusted = df_not_adjusted[df_not_adjusted['data_type_code'] == 'E_SM']

Rename the `cell_value` column to reflect its respective data type for clarity.

In [248]:
sm_df_adjusted = sm_df_adjusted.rename(columns={'cell_value': 'monthly_sales'})
esm_df_adjusted = esm_df_adjusted.rename(columns={'cell_value': 'monthly_sales_percent_change'})
sm_df_not_adjusted = sm_df_not_adjusted.rename(columns={'cell_value': 'monthly_sales'})
esm_df_not_adjusted = esm_df_not_adjusted.rename(columns={'cell_value': 'monthly_sales_percent_change'})

In [249]:
sm_df_not_adjusted['monthly_sales'].apply(float.is_integer).all()

np.True_

`monthy_sales` seem to be all rounded upto the whole number.

In [250]:
sm_df_not_adjusted['monthly_sales'] = sm_df_not_adjusted['monthly_sales'].astype(int)
sm_df_adjusted['monthly_sales'] = sm_df_adjusted['monthly_sales'].astype(int)

In [251]:
sm_df_not_adjusted.head()

,monthly_sales,category_code,seasonally_adjusted,data_type_code,time,category_name,high_level_category
0,92353,722,False,SM,2023-03-01,Food Services and Drinking Places,Food Services
1,92421,722,False,SM,2023-09-01,Food Services and Drinking Places,Food Services
2,8376,442,False,SM,2017-02-01,Furniture and Home Furnishings Stores,Retail Trade
3,10031,442,False,SM,2017-08-01,Furniture and Home Furnishings Stores,Retail Trade
4,24675,446,False,SM,2017-02-01,Health and Personal Care Stores,Retail Trade


In [252]:
esm_df_adjusted.head()

,monthly_sales_percent_change,category_code,seasonally_adjusted,data_type_code,time,category_name,high_level_category
1764,1,44X72,True,E_SM,2023-06-01,Retail Trade and Food Services,Retail and Food Services
1765,1,44X72,True,E_SM,2023-12-01,Retail Trade and Food Services,Retail and Food Services
1766,0,4521E,True,E_SM,2023-04-01,Department Stores,Retail Trade
1767,0,4521E,True,E_SM,2023-10-01,Department Stores,Retail Trade
1768,1,44000,True,E_SM,2017-02-01,Retail Trade,Retail Trade


In [253]:
output_directory = "../data/"

# Export DataFrames to CSV files
sm_df_adjusted.to_csv(f"{output_directory}sm_df_adjusted.csv", index=False)
esm_df_adjusted.to_csv(f"{output_directory}esm_df_adjusted.csv", index=False)
sm_df_not_adjusted.to_csv(f"{output_directory}sm_df_not_adjusted.csv", index=False)
esm_df_not_adjusted.to_csv(f"{output_directory}esm_df_not_adjusted.csv", index=False)